In [1]:
# run stopwatch
from tools import Stopwatch
stopwatch = Stopwatch()
start = stopwatch.start()

### Load libraries, functions, palette, theme

In [2]:
%run _libraries.ipynb

In [3]:
%run _functions.ipynb

In [4]:
save_files = 'files/'

In [5]:
save_img = 'docs/img/'

In [6]:
session_name = 'research'

# Section II. Models

## Load Saved Section if exists

## Load Data

In [8]:
train = datasets_dict = loadit(
    'train', dir=save_files)

In [9]:
valid = datasets_dict = loadit(
    'valid', dir=save_files)

In [10]:
test = datasets_dict = loadit(
    'test', dir=save_files)

## Model 1: SARIMAX

In [15]:
from pmdarima.arima import auto_arima

In [17]:
train.columns

Index(['temperature', 'humidity', 'windspeed', 'generaldiffuseflows',
       'diffuseflows', 'target1', 'target2', 'target3', 'minute', 'hour',
       'day', 'day_number', 'day_of_year', 'weekday', 'week', 'month',
       'year_month', 'year_day', 'count_elmnts_by_day', 'is_Friday',
       'is_Sunday', 'target1_diff', 'target1_returns', 'target1_volatility',
       'p', 'is_31', 'target1_cum_avg', 'target1_by_weekday'],
      dtype='object')

In [26]:
train_ = train[[
    'temperature', 'minute', 'hour', 'day', 'weekday',
    'is_Friday', 'is_Sunday', 'is_31', 'target1_cum_avg', 'target1_by_weekday', 'target1']].copy()

In [70]:
y = train_['target1'].copy()

In [71]:
y.shape

(17568,)

In [72]:
X = train_.iloc[:, :-1].copy()

In [73]:
X.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17568 entries, 2017-04-01 00:00:00 to 2017-07-31 23:50:00
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   temperature         17568 non-null  float64
 1   minute              17568 non-null  int32  
 2   hour                17568 non-null  int32  
 3   day                 17568 non-null  int32  
 4   weekday             17568 non-null  int32  
 5   is_Friday           17568 non-null  int64  
 6   is_Sunday           17568 non-null  int64  
 7   is_31               17568 non-null  int64  
 8   target1_cum_avg     17568 non-null  float64
 9   target1_by_weekday  17568 non-null  float64
dtypes: float64(3), int32(4), int64(3)
memory usage: 1.2 MB


In [74]:
X_ = np.asarray(X)

In [75]:
X_.shape

(17568, 10)

In [76]:
model_auto = auto_arima(y, X_)


KeyboardInterrupt

